Project: Combine input, if-else, and loops

Task: Build a mini billing system (subtotal, tax, total)

First, let's create our stock and define the cart

```
                                                  START
                                                    ↓
                                  Show stock items → Ask user for item
                                                    ↓
                                            Is item in stock?
                                          Yes ──────├────── No
                                          ↓                 ↓
                      Is item already in cart?          "Error 404"
                          |─────────────┐
                          |             NO
                          |             ↓
                          |         [New Item]  → ask for qty → add to cart
                          ↓
                         YES
          ┌─────────────┬───────────────┬
          ↓             ↓               ↓              
        [Continue]   [Adjust Qty]   [Check Out]      
          ↓             ↓               ↓              
      purchase()    ask for qty     Show subtotal    
          ↓             ↓               ↓              
        stock--     update cart    ask to pay?     
          ↓             ↓               ↓              
        ask:  (Continue or check out)  Exit
  continue shopping?
          ↓
      No ─├─ Yes → purchase()
      ↓           ↓
      ↓          END             
Show subtotal → ask to pay?



```



In [57]:
import pandas as pd

In [ ]:
stock = {
    "Apple": {"Stock": 5, "Price": 20},
    "Banana": {"Stock": 12, "Price": 17},
    "Mango": {"Stock": 10, "Price": 25}
}
cart = {}
# Convert the stock dictionary to a pandas DataFrame
stock_df = pd.DataFrame.from_dict(stock, orient='index')
# Save the DataFrame to a CSV file
stock_df.to_csv('stock_data.csv')

print("Stock data saved to a csv file")

In [58]:
TAX_RATE = 1.12

def show_items():
    print("Available items:")
    for name, info in stock.items():
        print(f"- {name}: {info['Stock']} in stock at ${info['Price']} each")
    print()


def get_valid_choice():
    while True:
        purchase = input("Choose an item: ").strip().title()
        if purchase in stock:
            if stock[purchase]["Stock"] > 0:
                return purchase
            else:
                print(f"Sorry, {purchase} is out of stock.")
        else:
            print("Item not found. Please choose from the list.")


def get_valid_amount(purchase):
    max_amount = stock[purchase]["Stock"]
    while True:
        amount_str = input(f"Enter quantity (1–{max_amount}): ").strip()
        if amount_str.isdigit():
            amount = int(amount_str)
            if 1 <= amount <= max_amount:
                return amount
        print("Invalid quantity; please try again.")


def calculate_subtotal():
    subtotal = sum(stock[item]["Price"] * data["amount"] for item, data in cart.items())
    return round(subtotal * TAX_RATE, 2)


def handle_new_item(purchase):
    amount = get_valid_amount(purchase)
    cart[purchase] = {"amount": amount}
    stock[purchase]["Stock"] -= amount
    print(f"Added {amount} x {purchase} to cart.")


def handle_existing_item(purchase):
    current = cart[purchase]["amount"]
    print(f"You have {current} x {purchase} in your cart.")
    while True:
        action = input("Choose: (1) Adjust quantity, (2) Continue shopping, (3) Checkout: ").strip()
        if action == '1':
            new_amount = get_valid_amount(purchase)
            diff = new_amount - current
            stock[purchase]["Stock"] -= diff
            cart[purchase]["amount"] = new_amount
            print(f"Updated {purchase} to {new_amount} units in cart.")
            return 'continue'
        elif action == '2':
            return 'continue'
        elif action == '3':
            return 'checkout'
        else:
            print("Invalid option; please enter 1, 2, or 3.")


def checkout():
    total = calculate_subtotal()
    print("\nCheckout Summary:")
    for purchase, data in cart.items():
        price = stock[purchase]["Price"]
        print(f"- {purchase}: {data['amount']} x ${price} = ${price * data['amount']}")
    print(f"Total with tax: ${total}\n")
    confirm = input("Proceed to payment? (y/n): ").strip().lower()
    if confirm == 'y':
        print("Payment successful. Thank you for shopping!")
        return True
    else:
        print("Checkout canceled; returning to shopping.")
        return False

# Save transaction to a csv file
def to_csv(cart):
  cart_df = pd.DataFrame.from_dict(cart, orient='index')
  cart_df.to_csv('cart_data.csv')

def main():
    print("Welcome to the shop!\n")
    while True:
        show_items()
        purchase = get_valid_choice()
        if purchase in cart:
            action = handle_existing_item(purchase)
        else:
            handle_new_item(purchase)
            action = input("Continue shopping or checkout? (c/checkout): ").strip().lower()
        if action in ('checkout', '3'):
            if checkout():
                to_csv(cart)
                break


if __name__ == '__main__':
    main()


Welcome to the shop!

Available items:
- Apple: 5 in stock at $20 each
- Banana: 12 in stock at $17 each
- Mango: 8 in stock at $25 each

Choose an item: apple
Enter quantity (1–5): 2
Added 2 x Apple to cart.
Continue shopping or checkout? (c/checkout): checkout

Checkout Summary:
- Mango: 2 x $25 = $50
- Apple: 2 x $20 = $40
Total with tax: $100.8

Proceed to payment? (y/n): y
Payment successful. Thank you for shopping!
